## Procesamiento Datos Pandas

Para esta prueba deberá emplear los datos del archivo candidates.csv para lo cual deberá:

1. Almanecar los datos en una base de datos. (Asegurar el almacenamiento con el tipo de dato apropiado)
2. A partir de la lectura de los datos deberá generar un procesamiento reporte.
    
    (Cree el proceso que consideré más conveniente, realizarlo lo más complejo que le sea posible)
    
3. Realizar un código de envio de correos integrado en su solución.


El archivo candidates.csv contiene 50k filas de datos sobre candidatos. Los campos que estamos usando son:
- First Name
- Last Name
- Email
- Country
- Application Date
- YOE
- Seniority
- Technology
- Code Challenge Score
- Technical Interview


In [9]:
import sqlite3
import pandas as pd

df_candidates = pd.read_csv('./src/candidates.csv', sep=';')

conn = sqlite3.connect('candidates.db')

df_candidates.to_sql('candidates', conn, if_exists='replace', index=False)


report = pd.read_sql_query('''
    SELECT Country, AVG(YOE) AS Average_YOE, AVG([Code Challenge Score]) AS Average_Score
    FROM candidates
    GROUP BY Country
    ORDER BY Average_Score DESC
''', conn)

conn.close()
print(report)

           Country  Average_YOE  Average_Score
0            Sudan    16.050000       5.633333
1           Tuvalu    15.015544       5.549223
2       Madagascar    15.296482       5.547739
3            Japan    15.511737       5.516432
4    Guinea-Bissau    15.404040       5.515152
..             ...          ...            ...
239       Colombia    15.389381       4.561947
240        Denmark    15.320755       4.547170
241     Guadeloupe    14.456221       4.520737
242      Hong Kong    14.797030       4.465347
243           Guam    15.630769       4.410256

[244 rows x 3 columns]


In [11]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders


de = "kelly.humpire21@gmail.com"
clave = open('clave.txt').read().strip()
para = "destinatario@gmail.com"


mensaje = MIMEMultipart()
mensaje["From"] = de
mensaje["To"] = para
mensaje["Subject"] = "Base de candidatos"

archivo_adjunto = "candidates.db" 
with open(archivo_adjunto, "rb") as archivo:
    adjunto = MIMEBase("application", "octet-stream")
    adjunto.set_payload(archivo.read())
encoders.encode_base64(adjunto)

adjunto.add_header("Content-Disposition", f"attachment; filename= {archivo_adjunto}")
mensaje.attach(adjunto)

smtp_server = 'smtp.gmail.com'
smtp_port = 587
with smtplib.SMTP(smtp_server, smtp_port) as server:
    server.starttls()
    server.login(de, clave)
    server.send_message(mensaje)

print('Correo electrónico enviado con éxito.')

Correo electrónico enviado con éxito.
